In [0]:
import os
import sys
# # print(os.getcwd())      
# project_pth = os.getcwd()
# sys.path.append(project_pth)

In [0]:
import importlib.util

module_name = "transformation"
file_path = "/Workspace/Users/omkarmhetre800@gmail.com/Spotify/utils/transformation.py"

spec = importlib.util.spec_from_file_location(module_name, file_path)
transformation = importlib.util.module_from_spec(spec)
sys.modules[module_name] = transformation
spec.loader.exec_module(transformation)

from transformation import *

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS spotify.silver_schema.dimuser_checkpoint;
CREATE VOLUME IF NOT EXISTS spotify.silver_schema.dimartist_checkpoint;
CREATE VOLUME IF NOT EXISTS spotify.silver_schema.dimtrack_checkpoint;
CREATE VOLUME IF NOT EXISTS spotify.silver_schema.dimdate_checkpoint;
CREATE VOLUME IF NOT EXISTS spotify.silver_schema.factstream_checkpoint;

In [0]:
df_user = (
    spark.readStream
         .table("spotify.bronze_schema.dimuser")
         .dropDuplicates(["user_id"])
)

(
    df_user.writeStream
        .format("delta")
        .outputMode("append")
        .option(
            "checkpointLocation",
            "/Volumes/spotify/silver_schema/dimuser_checkpoint"
        )
        .trigger(once=True)
        .toTable("spotify.silver_schema.dimuser")
)


In [0]:
df_art = (
    spark.readStream
         .table("spotify.bronze_schema.dimartist")
         .dropDuplicates(["artist_id"])
)

(
    df_art.writeStream
        .format("delta")
        .outputMode("append")
        .option(
            "checkpointLocation",
            "/Volumes/spotify/silver_schema/dimartist_checkpoint"
        )
        .trigger(once=True)
        .toTable("spotify.silver_schema.dimartist")
)

In [0]:
from pyspark.sql.functions import col, when, regexp_replace as re

df_track = (
    spark.readStream
         .table("spotify.bronze_schema.dimtrack")
)

df_track = ( df_track.withColumn("durationFlag",when(col('duration_sec')<150,"low")
                                .when(col('duration_sec')>350,"medium")
                                .otherwise("high")
                               )
            )

df_track = df_track.withColumn("track_name", re(col("track_name"), '-', ' '))

# display(
#     df_track,
#     checkpointLocation="dbfs:/Volumes/spotify/silver_schema/dimtrack_checkpoint"
# )

(
    df_track.writeStream
        .format("delta")
        .outputMode("append")
        .option(
            "checkpointLocation",
            "/Volumes/spotify/silver_schema/dimtrack_checkpoint"
        )
        .trigger(once=True)
        .toTable("spotify.silver_schema.dimtrack")
)

In [0]:
%sql
SELECT * FROM spotify.silver_schema.dimtrack limit 5;

In [0]:
df_date= (
    spark.readStream
         .table("spotify.bronze_schema.dimdate")
)

(
    df_art.writeStream
        .format("delta")
        .outputMode("append")
        .option(
            "checkpointLocation",
            "/Volumes/spotify/silver_schema/dimdate_checkpoint"
        )
        .trigger(once=True)
        .toTable("spotify.silver_schema.dimdate")
)

In [0]:
df_fact = (
    spark.readStream
         .table("spotify.bronze_schema.factstream")
)

# df_fact = reusable().dropColumns(df_fact,['_rescued_data']) 


(
    df_fact.writeStream
        .format("delta")
        .outputMode("append")
        .option(
            "checkpointLocation",
            "/Volumes/spotify/silver_schema/factstream_checkpoint"
        )
        .trigger(once=True)
        .toTable("spotify.silver_schema.dimfactstream")
)



In [0]:
%sql
DROP TABLE IF EXISTS spotify.silver_schema.dimdate;


In [0]:
dbutils.fs.rm(
    "dbfs:/Volumes/spotify/silver_schema/dimartist_checkpoint",
    recurse=True
)

dbutils.fs.rm(
    "dbfs:/Volumes/spotify/silver_schema/dimtrack_checkpoint",
    recurse=True
)


In [0]:
dbutils.fs.rm(
    "dbfs:/Volumes/spotify/silver_schema/dimdate_checkpoint",
    recurse=True
)